# Summary

This Notebook showcases the Demand Forecasting usecase by leveraging Cumulocity Machine Learning.

This usecase demonstration is divided into three steps:

    1. Extract the device measurement data from cumulocity operational store.
    2. Use Nyoka microservice to generate Timeseries model.
    3. Use Zementis microservice to generate forecast from the Timeseries model.

> Note: Run the above steps in sequence. Additionally, run the data simulation script (simulate_data.sh) only if the measurements from the device are not available. The measurement data gets cleaned up based on the retention period configured on Cumulocity tenant.

##### It is important to remember that forecast intervals will match the observation intervals. The simulation script, simulate_data.sh, generates water consumption data for every two hours for a period of 1 week. The forecasts generated from a Timeseries model trained on this data will be for every two hour in the future.

## 1. Extract the device measurement data from cumulocity operational store

This step extracts the measurements of the device configured in CONFIG.INI file. It uses the following REST API call:
    
[https://{tenant-url}/measurement/measurements](https://{tenant-url}/measurement/measurements)
    
The measurement data is used as training data set for generating the Timeseries model in the next step. 

In [ ]:
import requests
from requests.auth import HTTPDigestAuth
import json
from jsonpath_ng import jsonpath, parse
import time
import pandas as pd
from pandas import read_csv
import csv
from datetime import datetime as pdt,timedelta, date
import os
import matplotlib as inline
import datetime
import configparser
import sys
import shutil


config = configparser.ConfigParser()
config.read('CONFIG.ini')
tenant_url =config['cumulocity']['c_url']
user_name=config['cumulocity']['c_user']
password=config['cumulocity']['c_pass']
tz_zone_info='0530'

deviceID=config['cumulocity']['c_device_source']

#start Date
stDate="2019-12-01"
#duration
duration=7 
training_start_date=''

stDate=stDate+"T00:00:00+"+tz_zone_info

START_DATE= datetime.datetime.strptime(stDate, "%Y-%m-%dT%H:%M:%S%z").date()
END_DATE=START_DATE+timedelta(days=int(duration))

endDate=datetime.datetime.strftime(END_DATE,"%Y-%m-%d")

c8y_Flow=list()
measurement_times=list()

if not os.path.exists('temp'):
    os.mkdir("temp")

def plot_the_predicted_values():
    
    from matplotlib import pyplot
    import matplotlib 
    %matplotlib inline 
    series = read_csv('temp/input_data_for_training.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    series.plot()
    print('\nPlotting the ## c8y_Flow ## Measurements of the device ## '+deviceID +' ##')
    pyplot.show()


def extract_measurements(responseContent):
    jData = json.loads(responseContent)
    jsonpath_expression = parse('$.measurements[*].c8y_Flow.F.value')
    for match in jsonpath_expression.find(jData):
        c8y_Flow.append(float(f'{match.value}'))
        
    jsonpath_expression = parse('$.measurements[*].time')
    for match in jsonpath_expression.find(jData):
        measurement_times.append(f'{match.value}')
    training_start_date_expr = parse('$.measurements[0].time')
    for match1 in training_start_date_expr.find(jData):
        global training_start_date
        training_start_date = str(f'{match1.value}')
        
def draw_measurements_train_model():
    # Replace with the correct URL
    URL = tenant_url+'/measurement/measurements'

    PARAMS = {'dateTo':str(END_DATE)+"T23:59:00+"+tz_zone_info,
             'dateFrom':str(START_DATE)+"T00:00:00+"+tz_zone_info,
             'source':deviceID, 
             'withTotalPages':'true',
             'pageSize':'2000'} 
    print('\nCollecting ## c8y_Flow ## measurements of the device ## '+ deviceID +' ## for the duration of - '+str(START_DATE)+' and '+str(END_DATE) +' from Cumulocity')

    response_content = requests.get(url = URL, params = PARAMS, auth=(user_name, password), verify=True) 

    # For successful API call, response code will be 200 (OK)
    if response_content.ok:
        jContent=json.loads(response_content.content)
        extract_measurements(response_content.content)
        if 'next' in jContent:
            nextPage=jContent['next']
            while nextPage != None and len(nextPage) != 0:
                response_content=get_measurement_request(nextPage)
                extract_measurements(response_content.content)
                jsonData=json.loads(response_content.content)
                if 'next' in jsonData:
                    nextPage=jsonData['next']
                else:
                    nextPage=''
    else:
        response_content.raise_for_status();
    
    if len(c8y_Flow)==0:
        print("\nc8Y_Flow values are empty.Please follow the setup instructions before running the script.")
        return
    print('\nExtracted ## c8y_Flow ## Measurements of the device ## '+deviceID +' ##')
    
    datetimes = []
    for x in measurement_times:
        tmp_time=pdt.strptime(x.split('+')[0], "%Y-%m-%dT%H:%M:%S.%f")
        datetimes.append(pd.Timestamp(tmp_time))
    df=pd.DataFrame(data=c8y_Flow,index=pd.DatetimeIndex(datetimes))
    df.to_csv('temp/input_data_for_training.csv')
    plot_the_predicted_values()

    
draw_measurements_train_model()

## 2. Use Nyoka microservice to generate TimeSeries model

In this step, the extracted measurement data is sent to Nyoka microservice for training a Timeseries model using the following REST API. This API returns the name of the model that is being trained and its status.

[https://{tenant_url}/service/zementis/timeseries](https://{tenant_url}/service/zementis/timeseries)

Nyoka Microservice generates a Timeseries model with the submitted training data and deploys it to the Zementis microservice. We can also track the status of the Timeseries model generation with following REST API.

[https://{tenant_url}/service/zementis/timeseries/{model_name}/status](https://{tenant_url}/service/zementis/timeseries/{model_name}/status)


In [ ]:
data={}
time_series_data_for_training=''
uploaded_model=''
def prepare_time_series_data_for_training():
    df= pd.read_csv('temp/input_data_for_training.csv',index_col=0)
    data['startDate']=training_start_date #str(START_DATE)+'T00:00:00+05:30'
    startDate = pdt.strptime(str(START_DATE)+'T00:00:00', "%Y-%m-%dT%H:%M:%S")
    datetimes = []
    last_date_time = startDate
    data['series'] = df.values.flatten().tolist()
    interval={}
    interval['timeUnit']='HOURS'
    interval['periodLength']=2
    data['observationInterval']=interval
    seasonality={}
    seasonality['timeUnit']='HOURS'
    seasonality['periodLength']=24
    data['seasonality']=seasonality
    time_series_data = json.dumps(data)
    print('\nPreparing the Extracted Measurements as per Timeseries API specification.')
    return time_series_data;

def get_exception_msg(resp_text):
    resp_json=json.loads(resp_text)
    if 'error' in resp_json:
        return resp_json['error']
    elif 'errors' in resp_json:
        return resp_json['errors']

def train_and_generate_timeseries_model():
    time_series_data_for_training=prepare_time_series_data_for_training()
    print('\nThe extracted  data is sent to Nyoka Microservice for training to get the besfit model\n')
    print(time_series_data_for_training)

    postURL=tenant_url+"/service/zementis/timeseries"
    post_resp=requests.post(url=postURL,data=time_series_data_for_training,headers = {'content-type': 'application/json'},auth=(user_name, password),verify=True)
    if not post_resp.ok:
        print('\n'+str(get_exception_msg(post_resp.text)));
        sys.exit()

    resp_map=json.loads(post_resp.content) 
    global uploaded_model
    uploaded_model = resp_map['modelName']

    print('\nThe TimeSeries Model generated by Nyoka Microservice--'+uploaded_model+'.Zementis microservice is waiting for it.')

    model_uploaded_status_url= resp_map['statusUrl']

    status_resp =requests.get(url=tenant_url+model_uploaded_status_url,auth=(user_name, password),verify=True)

    status_resp_map = json.loads(status_resp.content)

    while 'status' in status_resp_map and status_resp_map['status'] == 'IN_PROGRESS':
        print('\n'+uploaded_model + ' Model is not yet avaialble to Zementis Micro Service.....\n')
        time.sleep(50)
        status_resp =requests.get(url=tenant_url+model_uploaded_status_url,auth=(user_name, password),verify=True)
        status_resp_map = json.loads(status_resp.content)
        
    print('The ##'+uploaded_model + '## is now available for Zementis microservice') 
    
train_and_generate_timeseries_model()

## 3. Use Zementis microservice to generate forecast from the Timeseries model

In this step, we generate the forecast for the next day (next 12 values) and plot the forecasted data points along with training data points. Forecasting uses the following REST API:
    
[https://{tenant_url}/service/zementis/apply/{model_name}/record={"h":{value}}](https://{tenant_url}/service/zementis/apply/{model_name}/record={"h":{value}})
     
> h is the query parameter which represents number of time steps ahead to forecast.

In [ ]:
def plot_the_predicted_values():
    
    from matplotlib import pyplot
    import matplotlib 
    %matplotlib inline 
    series = read_csv('temp/input_data_for_training.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    series.plot()
    series1 = read_csv('temp/predict.csv', header=0, index_col=0, parse_dates=True, squeeze=True)
    series1.plot()
    pyplot.show()
    
    testseries =read_csv('temp/predict.csv',skiprows=[1,1])
    testseries=testseries.sort_values(by=["predicted_value"])
    print(testseries)
    
    peak_hour=testseries.iloc[0]["Date"]
    peak_consumption=testseries.iloc[0]["predicted_value"]
    
    row_count=len(testseries)
    min_hour=testseries.iloc[row_count-1]["Date"]
    min_consumption=testseries.iloc[row_count-1]["predicted_value"]
    
    print('\n The Peak Consumption is Forecasted at '+str(peak_hour)+' with the value :: '+str(peak_consumption) )
    print('\n The Minimum Consumption Forecasted at '+str(min_hour)+' with the value :: '+str(min_consumption) )
    if os.path.exists('temp'):
        shutil.rmtree('temp')

def run_the_predictions_for_next_day():
    # Run the Inference for the next day with 48 numbers
    #Preparing input csv for next day
    next_day_date=END_DATE+timedelta(days=1)
    
    print('Using ##'+uploaded_model + '## for prediction.')
    print('\nRunning the Predictions for the Next day '+str(next_day_date)+'i.e 12 future values')
    
    #Calling timeseries for predictions
    url = tenant_url+'/service/zementis/apply/'+uploaded_model
    prediction_response=requests.get(url = url, params = 'record={"h":12}', auth=(user_name, password), verify=True) 
    
    if prediction_response.ok:
        prediction_response=json.loads(prediction_response.text)
    else:
        print("\n Exception Occured: "+prediction_response.text);
        return
    
    training_data=read_csv('temp/input_data_for_training.csv')
    num_rows=len(training_data.index)
    last_reading_date=training_data.iloc[num_rows-1][0]
    last_reading_date=last_reading_date.split(' ')

    eDate = pdt.strptime(last_reading_date[0]+'T'+last_reading_date[1], "%Y-%m-%dT%H:%M:%S")
    
    prd=list()
    prd.append(training_data.iloc[num_rows-1][1])
    predictions_dict= prediction_response["outputs"][0]
    for key in range(1,13):
        prd.append(predictions_dict["predicted_value"].get(str(key)))
    
    
    datetimes = []
    for _ in prd:
        datetimes.append(pd.Timestamp(eDate))
        eDate = eDate+timedelta(hours=2)
    df=pd.DataFrame(data=prd,index=pd.DatetimeIndex(datetimes))
    df.index.name='Date'
    df.to_csv('temp/predict.csv',header=[ 'predicted_value'])
    
    plot_the_predicted_values()

run_the_predictions_for_next_day()
